In [ ]:
MORNINGSTAR_INDUSTRY_GROUP_CODES_after_Nov2019 =  {10110: 'Agricultural Inputs',
 10120: 'Building Materials',
 10130: 'Chemicals,Specialty Chemicals',
 10140: 'Lumber & Wood Production,Paper & Paper Products',
 10150: 'Aluminum,Copper,Other Industrial Metals & Mining,Gold,Silver,Other Precious Metals & Mining',
 10160: 'Coking Coal,Steel',
 10200: 'Auto & Truck Dealerships,Auto Manufacturers,Auto Parts,Recreational Vehicles',
 10220: 'Furnishings, Fixtures & Appliances',
 10230: 'Residential Construction',
 10240: 'Textile Manufacturing,Apparel Manufacturing,Footwear & Accessories',
 10250: 'Packaging & Containers',
 10260: 'Personal Services',
 10270: 'Restaurants',
 10280: 'Apparel Retail,Department Stores,Home Improvement Retail,Luxury Goods,Internet Retail,Specialty Retail',
 10290: 'Gambling,Leisure,Lodging,Resorts & Casinos,Travel Services',
 10310: 'Asset Management',
 10320: 'Banks—Diversified,Banks—Regional,Mortgage Finance',
 10330: 'Capital Markets,Financial Data & Stock Exchanges',
 10340: 'Insurance—Life,Insurance—Property & Casualty,Insurance—Reinsurance,Insurance—Specialty,Insurance Brokers,Insurance—Diversified',
 10350: 'Shell Companies,Financial Conglomerates',
 10360: 'Credit Services',
 10410: 'Real Estate—Development,Real Estate Services,Real Estate—Diversified',
 10420: 'REIT—Healthcare Facilities,REIT—Hotel & Motel,REIT—Industrial,REIT—Office,REIT—Residential,REIT—Retail,REIT—Mortgage,REIT—Specialty,REIT—Diversified',
 20510: 'Beverages—Brewers,Beverages—Wineries & Distilleries',
 20520: 'Beverages—Non-Alcoholic',
 20525: 'Confectioners,Farm Products,Household & Personal Products,Packaged Foods',
 20540: 'Education & Training Services',
 20550: 'Discount Stores,Food Distribution,Grocery Stores',
 20560: 'Tobacco',
 20610: 'Biotechnology',
 20620: 'Drug Manufacturers—General,Drug Manufacturers—Specialty & Generic',
 20630: 'Healthcare Plans',
 20645: 'Medical Care Facilities,Pharmaceutical Retailers,Health Information Services',
 20650: 'Medical Devices,Medical Instruments & Supplies',
 20660: 'Diagnostics & Research',
 20670: 'Medical Distribution',
 20710: 'Utilities—Independent Power Producers,Utilities—Renewable',
 20720: 'Utilities—Regulated Water,Utilities—Regulated Electric,Utilities—Regulated Gas,Utilities—Diversified',
 30810: 'Telecom Services',
 30820: 'Advertising Agencies,Publishing,Broadcasting,Entertainment',
 30830: 'Internet Content & Information,Electronic Gaming & Multimedia',
 30910: 'Oil & Gas Drilling,Oil & Gas E&P,Oil & Gas Integrated,Oil & Gas Midstream,Oil & Gas Refining & Marketing,Oil & Gas Equipment & Services',
 30920: 'Thermal Coal,Uranium',
 31010: 'Aerospace & Defense',
 31020: 'Specialty Business Services,Consulting Services,Rental & Leasing Services,Security & Protection Services,Staffing & Employment Services',
 31030: 'Conglomerates',
 31040: 'Engineering & Construction,Infrastructure Operations,Building Products & Equipment',
 31050: 'Farm & Heavy Construction Machinery',
 31060: 'Industrial Distribution',
 31070: 'Business Equipment & Supplies,Specialty Industrial Machinery,Metal Fabrication,Pollution & Treatment Controls,Tools & Accessories,Electrical Equipment & Parts',
 31080: 'Airports & Air Services,Airlines,Railroads,Marine Shipping,Trucking,Integrated Freight & Logistics',
 31090: 'Waste Management',
 31110: 'Information Technology Services,Software—Application,Software—Infrastructure',
 31120: 'Communication Equipment,Computer Hardware,Consumer Electronics,Electronic Components,Electronics & Computer Distribution,Scientific & Technical Instruments',
 31130: 'Semiconductor Equipment & Materials,Semiconductors,Solar'}

# 第一波で大きく下落した銘柄とセクターを調べる

+ 第一波は，SPYの年初来最安値3/23にする
+ 年初来から2020-06-19までの値動きを，QTradableStocksUS で探し，セクター毎に確認


In [ ]:
import numpy as np

from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.morningstar import Fundamentals
from quantopian.pipeline.domain import US_EQUITIES
from quantopian.research import run_pipeline
from quantopian.pipeline.filters import Q500US, StaticSids, StaticAssets
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline.factors import Returns

from quantopian.pipeline.classifiers.fundamentals import Sector


def make_pipeline():
    
    base_universe = QTradableStocksUS()
    close_price = USEquityPricing.close.latest

    morningstar_industry_group_code = Fundamentals.morningstar_industry_group_code.latest
    prev_change = Returns(inputs = [USEquityPricing.close], window_length=2)
    financial_health_grade = Fundamentals.financial_health_grade.latest
    
    
    
    pipe = Pipeline(
        columns={
            'morningstar_industry_group_code':morningstar_industry_group_code, 
            'close_price':close_price, 
            "prev_change": prev_change,
            'financial_health_grade':financial_health_grade,
        },
        
        screen=base_universe,
    )
    return pipe 

df = run_pipeline(make_pipeline(), '2020-01-01', '2020-06-18')
df["sector_name"] = df["morningstar_industry_group_code"].replace(MORNINGSTAR_INDUSTRY_GROUP_CODES_after_Nov2019)

df.head()

In [ ]:
df.groupby("morningstar_industry_group_code")["prev_change"].sum().sort_values().plot(kind="bar")



In [ ]:
df.groupby(by = "sector_name")[["prev_change"]].sum().sort_values(by="prev_change").head()

## セクター毎に値動きを確認

In [ ]:
import pandas as pd 

d = dict()

for sector, x in df.groupby(by = "morningstar_industry_group_code")["prev_change"]:
    d[sector] = x.unstack().cumsum().mean(axis=1)
    
df_by_sector = pd.DataFrame(d)
df_by_sector = df_by_sector.drop(-1, axis=1)
df_by_sector.head()

In [ ]:
sorted_by_worse = df_by_sector.min().sort_values().index


In [ ]:
x = df_by_sector.columns
n = len(x)
yoko = 5
tate = int(n / yoko) + 1
print(tate,yoko)

df_by_sector[sorted_by_worse].plot(subplots=True, layout=(tate, yoko), figsize=(20,20))


## 観察１

+ 30830：ゲーム関連強すぎる
+ 30910：オイル，ガスは，ちょうどロシアVSサウジの喧嘩も相まって，原油がマイナスに入ったりもしたので，この状態になった．ただしこの中に入っている，タンカー銘柄やガス銘柄のパフォーマンスはとても良かった
+ 10420: 'REIT—Healthcare Facilities,REIT—Hotel & Motel,REIT—Industrial,REIT—Office,REIT—Residential,REIT—Retail,REIT—Mortgage,REIT—Specialty,REIT—Diversified',：商業不動産（ショッピングモール系）はガタガタだが，オフィスやレジデンス系は落ちていない．
+ 20670: 'Medical Distribution' つよい
+ 31030 はCクラスの一社のみなので無視
+ 10350 も2社しかないので無視


# 2波のボトムで買いたい 

## 20670 Medical Distribusion 

![](https://i.imgur.com/aKPFroW.png)

+ [ABC](https://finance.yahoo.com/quote/ABC/profile?p=ABC)
    + 米国および世界各国で医薬品の仕入・販売を行っています。医薬品流通部門は、急性期病院や医療システム、独立系・チェーン系の小売薬局、通信販売薬局、診療所、長期療養型病院、その他の代替施設薬局、その他の顧客を含む様々な医療提供者に、ブランド医薬品、ジェネリック医薬品、一般用医薬品、在宅医療用品・機器、外部委託された無菌製剤、および関連サービスを提供しています。また、薬局管理、スタッフィング、その他のコンサルティングサービス、小売・施設向けサプライマネジメントソフトウェア、各種施設・小売・施設向け包装ソリューションを提供しています。また、血漿などの血液製剤、注射剤、ワクチンなどのスペシャリティ製品の販売、がんを中心とした各種疾患を専門とする医師や病院、透析クリニックなどの医療機関を対象としたその他のサービスの提供、バイオテクノロジーや製薬メーカー向けのデータ分析、アウトカムリサーチ、その他のサービスの提供などを行っています。

##  20510: 'Beverages—Brewers,Beverages—Wineries & Distilleries',

![](https://i.imgur.com/q1Go1tL.png)


+ [BF-B](https://finance.yahoo.com/quote/BF-B/profile?p=BF-B)
    + ブラウン・フォーマン社は、世界中で各種アルコール飲料の製造、瓶詰め、輸入、輸出、販売、販売を行っています。スピリッツ、ワイン、ウイスキースピリッツ、ウイスキーベースのフレーバーリキュール、すぐに飲める製品とすぐに飲める製品、すぐに飲めるカクテル、ウォッカ、テキーラ、シャンパン、ブランデー、バーボン、リキュールを提供している。主にジャックダニエルズ、ウッドフォードリザーブ、カナディアンミスト、グレンドラナッハ、ベンリアック、グレングラスソー、オールドフォレスター、アーリータイムズ、スレーンアイリッシュウイスキー、クーパーズクラフト、アーリータイムズ、エル・ジマドール、ヘラデュラ、ニューミックス、ペペ・ロペス、アンティグオ、フィンランディア、コルベルシャンパン、ソノマ・キュートラーのブランドで製品を提供している。


## 20520 Beverages—Non-Alcoholic

![](https://i.imgur.com/sUS35Xx.png)

+ [FIZZ](https://finance.yahoo.com/quote/FIZZ/profile?p=FIZZ)
    + National Beverage Corp.は、子会社を通じて、主に米国およびカナダにおいて、水、ジュース、エナジードリンク、炭酸ソフトドリンクの開発、製造、販売、ポートフォリオの販売を行っています。同社は、「LaCroix」、「LaCroix Cúrate」、「LaCroix NiCola」、「Shasta Sparkling Water」、「Rip It」、「Everfresh」、「Everfresh Premier Varietals」、「Mr.Pure」ブランドのスパークリングウォーター、エナジードリンク、ジュース、「Shasta」、「Faygo」、「Red Pop」、「Moon Mist」、「Rock'n'Rye」ブランドの炭酸ソフトドリンクなど、アクティブで健康志向の高い消費者向けの飲料を提供しています

## 20525 Confectioners,Farm Products,Household & Personal Products,Packaged Foods

![](https://i.imgur.com/XwiabGu.png)


+ [CLX 214.59 2.59 1.22% : Clorox Company (The) - Yahoo Finance](https://finance.yahoo.com/quote/CLX/profile?p=CLX)
    + Cloroxブランドの漂白剤製品やClorox 2のステインファイター、カラーブースターなどの洗濯用添加剤、主にClorox、Formula 409、Liquid-Plumr、Pine-Sol、S.O.S.、Tilexブランドのホームケア製品、Green Worksブランドの天然由来製品、CloroxPro、Dispatch、Clorox Healthcare、Hidden Valley、KC Masterpieceブランドのプロ用クリーニング、消毒、フードサービス製品などを提供
+ [FRPT 85.83 0.06 0.07% : Freshpet, Inc. - Yahoo Finance](https://finance.yahoo.com/quote/FRPT/profile?p=FRPT)
    + フレッシュペット社は、米国、カナダ、英国において、犬や猫用の天然生鮮食品、冷蔵食、おやつを製造・販売
    
## 30830 Internet Content & Information,Electronic Gaming & Multimedia


![](https://i.imgur.com/zYhMWx1.png)
![](https://i.imgur.com/vunmDdY.png)
+ [EVER 60.94 2.34 3.99% : EverQuote, Inc. - Yahoo Finance](https://finance.yahoo.com/quote/EVER/profile?p=EVER)
    + 米国で保険ショッピングのためのオンラインマーケットプレイスを運営しています。同社のオンラインマーケットプレイスでは、自動車保険、住宅保険、賃貸住宅保険、生命保険、健康保険、商業保険のショッピングを提供しています。
+ [ANGI 12.72 0.49 4.01% : ANGI Homeservices Inc. - Yahoo Finance](https://finance.yahoo.com/quote/ANGI/profile?p=ANGI)
    + ANGI Homeservices Inc.は、住宅サービスのデジタルマーケットプレイスを運営しており、消費者と住宅サービスの専門家を結びつけています。同社のデジタルマーケットプレイスサービス「HomeAdvisor」は、住宅の修理、メンテナンス、改善プロジェクトのためのサービスの専門家と消費者を結びつけ、消費者に地元の、事前にスクリーニングされた、顧客評価の高いサービスの専門家を見つけるためのツールやリソースを提供するとともに、オンラインでの予約を提供しています。同社はまた、様々なサービスカテゴリーのサービス専門家のオンラインディレクトリーを通じて、消費者と地域のサービスの専門家を結びつける「Angie's List」を所有・運営しており、消費者が地域のサービスを調査・購入・採用するのに役立つ価値あるツール、サービス、検証済みレビューを含むコンテンツを提供しています。


## 20510 Beverages—Brewers,Beverages—Wineries & Distilleries

![](https://i.imgur.com/SDTlEdk.png)

+ [SAM 533.90 -2.88 -0.54% : Boston Beer Company, Inc. (The) - Yahoo Finance](https://finance.yahoo.com/quote/SAM?p=SAM&.tsrc=fin-srch)
    + ボストン・ビア・カンパニー社は、主に米国でアルコール飲料を製造・販売しています。同社の主力ビールは「サミュエルアダムス ボストンラガー」。
    

![](https://i.imgur.com/c1ombqu.png)

+ [TW 63.66 0.49 0.78% : Tradeweb Markets Inc. - Yahoo Finance](https://finance.yahoo.com/quote/TW/profile?p=TW)
    + 米国および国際的に電子マーケットプレイスを構築・運営しています。同社のマーケットプレイスは、金利、クレジット、マネーマーケット、株式など、さまざまな資産クラスの取引を容易にします。
    
+ [MKTX 500.86 -8.13 -1.60% : MarketAxess Holdings, Inc. - Yahoo Finance](https://finance.yahoo.com/quote/MKTX/profile?p=MKTX)
    + MarketAxess Holdings Inc.は、子会社とともに、債券市場の参加者が世界中で社債やその他の種類の債券を取引できるようにする電子取引プラットフォームを運営しています。機関投資家や証券会社に、米国の投資適格社債、新興市場およびハイイールド債、ユーロ債、米国エージェンシー債、地方債、レバレッジド・ローン、その他の債券のグローバルな流動性へのアクセスを提供しています。
    

## 10290 Gambling,Leisure,Lodging,Resorts & Casinos,Travel Services

一人だけ毛並みの違う子がいた．

![](https://i.imgur.com/sdizzkl.png)

+ [PTON 51.08 1.91 3.88% : Peloton Interactive, Inc. - Yahoo Finance](https://finance.yahoo.com/quote/PTON/profile?p=PTON)
    + 10290 のなかでもなぜかDに入っているこのひと
    + Peloton Interactive, Inc.は、北米でインタラクティブなフィットネス製品を提供しています。Peloton BikeやPeloton Treadなどのコネクテッド・フィットネス製品を提供しています。また、複数世帯のユーザーを対象としたコネクテッドフィットネスのサブスクリプションや、すべてのライブクラスやオンデマンドクラスへのアクセスを提供するコネクテッドフィットネスのサブスクライバー向けアプリ「Peloton Digital」も提供している。会員数は約140万人。

もう一人，いた

![](https://i.imgur.com/Y3WvBQe.png)


+   [VSTO 12.62 0.50 4.13% : Vista Outdoor Inc. - Yahoo Finance](https://finance.yahoo.com/quote/VSTO/profile?p=VSTO)
    + Vista Outdoor Inc.は、米国および海外のアウトドアスポーツおよびレクリエーション市場向けの消費者向け製品の設計、製造、販売を行っています。同社は、シューティング スポーツとアウトドア製品の 2 つのセグメントで事業を展開しています
    

## 10270 Restaurants

レベルが高い外食は潰れないよ．

![](https://i.imgur.com/6E1onH3.png)

+ [Chipotle Mexican Grill, Inc. (CMG) Stock Price, Quote, History & News - Yahoo Finance](https://finance.yahoo.com/quote/CMG?p=CMG&.tsrc=fin-srch)
+ [WING 120.67 -4.52 -3.61% : Wingstop Inc. - Yahoo Finance](https://finance.yahoo.com/quote/WING/profile?p=WING)
    + レストランをフランチャイズ展開しています。同社のレストランでは、クラシックな手羽先、骨なし手羽先、テンダーなどをオーダーメイドで調理し、手でソースをかけて様々な味付けでトスした手羽先を提供
+ [Yum China Holdings, Inc. (YUMC) Stock Price, Quote, History & News - Yahoo Finance](https://finance.yahoo.com/quote/YUMC?p=YUMC&.tsrc=fin-srch)
    + あーYuｍって中国の会社だったのか．NYSE上場．これ，どうなるんだろう．おもしろい．
    + Yum China Holdings, Inc.は、中国でレストランを所有、運営、フランチャイズ展開しています。同社はKFCとピザハットの2つのセグメントで運営している。KFC、Pizza Hut、Taco Bell、East Dawning、Little Sheep、COFFii & JOYのブランドでレストランを運営しており、チキン、ピザ、鍋料理、中華料理、メキシカン料理、コーヒーの各カテゴリーに特化している。また、オンラインフードデリバリーサービスも提供している。KFC」、「ピザハット」、「タコベル」、「リトルシープ」、「イーストドーン」、「COFFii & JOY」の名称でフランチャイズレストランを運営している。2020年3月31日現在、1,400都市で約9,295店舗を運営している。Yum China Holdings, Inc.は、中国石油化工股份有限公司および中国国家石油公司と戦略的契約を締結しており、ガソリンスタンドでのフランチャイズレストランの共同開発を行っている。Yum China Holdings, Inc.は2016年に法人化され、中国上海に本社を置いています。
    
    
![](https://i.imgur.com/FiYQw6d.png)

+ [PZZA 82.50 -0.38 -0.46% : Papa John's International, Inc. - Yahoo Finance](https://finance.yahoo.com/quote/PZZA/profile?p=PZZA)
    + パパ・ジョンズの商標のもとにピザのデリバリーとキャリーアウトのレストランを運営し、フランチャイズ化
    + デリバリーが上手く言ったのかもしれないので，2波で下がった場合は買いかもしれん．




# 第一波の時に大きくさげ，今戻している銘柄．


## 10290 Gambling,Leisure,Lodging,Resorts & Casinos,Travel Services


![](https://i.imgur.com/Y3WvBQe.png)

+ [PENN 31.73 -0.73 -2.23% : Penn National Gaming, Inc. - Yahoo Finance](https://finance.yahoo.com/quote/PENN/profile?p=PENN)
    + Penn National Gaming, Inc.は、子会社とともに、ゲーミング施設やレース施設を所有・管理し、スロットマシンを中心としたビデオゲーミング端末を運営
    
+ [BC 59.98 -0.91 -1.49% : Brunswick Corporation - Yahoo Finance](https://finance.yahoo.com/quote/BC/profile?p=BC)
    + Brunswick Corporationは、世界中のレクリエーション製品の設計、製造、販売を行っています。同社は、船外機、スターンドライブ、船内機および推進システムを含む船舶用エンジン製品、船舶用電子機器および制御統合システム、操舵システム、計器類、制御装置、プロペラ、トローリングモーター、燃料システム、電気システム、サービスパーツ、潤滑油、統合推進システムなどの船舶用エンジン製品を、レクリエーションおよび商業用船舶市場に提供しているほか、部品や付属品も提供しています。同社は、マーキュリー、マーキュリー・マークルーザー、マーキュリー・レーシング、モーターガイドの各ブランドのもと、海洋ディーラーや販売店、海洋専門店、海洋サービスセンターを通じて、独立系のボートビルダーや政府機関に舶用エンジン製品を提供しています。

+ [TRWH 23.45 -0.41 -1.72% : Twin River Worldwide Holdings, - Yahoo Finance](https://finance.yahoo.com/quote/TRWH/profile?p=TRWH)
    + ツインリバー・ワールドワイド・ホールディングスは、米国にゲーミングおよびレース施設を所有・運営しています。同社のゲーミングおよびレース施設には、スロットマシンや各種カジノテーブルゲーム、レストランおよびホテル施設が含まれます。
    
## 10270 Restaurants

C/Dクラスは文句なしで売りではないでしょうか？

Dクラス

![](https://i.imgur.com/StDM7tE.png)


Cクラス（最近上がった5社のみ）

![](https://i.imgur.com/CSZRcRA.png)

+ [JACK 72.29 0.85 1.19% : Jack In The Box Inc. - Yahoo Finance](https://finance.yahoo.com/quote/JACK/profile?p=JACK)
    + 2019年9月29日現在、21州とグアムで約2,243店舗のジャック・イン・ザ・ボックスレストランを運営・フランチャイズ化している。同社は1951年に設立され、カリフォルニア州サンディエゴに拠点
    

+ [WEN 22.05 0.23 1.05% : Wendy's Company (The) - Yahoo Finance](https://finance.yahoo.com/quote/WEN/profile?p=WEN)
    + ハンバーガーサンドに特化したクイックサービスレストランのシステムの運営、開発、フランチャイズ化に取り組んでいる。2019年12月29日現在、同社は約357店舗の自社所有レストラン、米国内で5,495店舗のフランチャイズレストラン、国際的に936店舗のフランチャイズレストランを運営している。また、不動産物件の所有・賃貸も行っている
    
## 10240 Textile Manufacturing,Apparel Manufacturing,Footwear & Accessories


![](https://i.imgur.com/9k1nguP.png)

+ [CROX 32.81 -0.20 -0.61% : Crocs, Inc. - Yahoo Finance](https://finance.yahoo.com/quote/CROX/profile?p=CROX)
    + クロックスは、下駄、サンダル、フリップス、スライド、靴、ブーツなど、様々なフットウェア製品をクロックスブランド名で提供
    + この下がり方は尋常じゃないな．戻りも凄いけど，凄いさがったな．
    
    
## 10230 Residential Construction

D クラス
![](https://i.imgur.com/n2Dtqoy.png)
+ [TMHC 18.80 -0.96 -4.86% : Taylor Morrison Home Corporatio - Yahoo Finance](https://finance.yahoo.com/quote/TMHC/profile?p=TMHC)
    + その子会社とともに、米国の公共住宅建設業者として事業を展開しています。同社は、一戸建ておよび複数世帯の付属・戸建住宅の設計・建設・販売、ライフスタイルやマスタープランド・コミュニティの開発を行っています。また、タイトル保険やクロージング決済サービス、金融サービスも提供している

Cクラス（最近上げてるTOP2)
![](https://i.imgur.com/7PJr4wZ.png)
+ [KB Home (KBH) Company Profile & Facts - Yahoo Finance](https://finance.yahoo.com/quote/KBH/profile?p=KBH)
    + 西海岸、南西部、中央部、南東部の4つのセグメントで事業を展開している。同社は、主に初めての住宅購入者、初入居者、初入居者、再入居者、アクティブな成人の住宅購入者を対象に、一戸建て・戸建住宅、タウンホーム、コンドミニアムなど、様々な住宅の建設・販売
    
+ [CCS 30.36 -1.25 -3.95% : Century Communities, Inc. - Yahoo Finance](https://finance.yahoo.com/quote/CCS/profile?p=CCS)
    + 一戸建ておよび一戸建て住宅の設計、開発、建設、マーケティング、販売を行っています。また、土地の権利化と開発、住宅購入者への抵当権、権原、保険サービス


In [ ]:

from quantopian.research import prices, symbols

def get_tickers_by_ranking(df, label):
    df_rank = df[df["financial_health_grade"] == label]
    tickers = df_rank.index
    return tickers 



sector = 10230

print(MORNINGSTAR_INDUSTRY_GROUP_CODES_after_Nov2019[sector])

lastdate = df.index.get_level_values(0)[-1]
df_lastdate = df[df["morningstar_industry_group_code"]==sector].loc[lastdate]
A = get_tickers_by_ranking(df_lastdate, "A")
B = get_tickers_by_ranking(df_lastdate, "B")
C = get_tickers_by_ranking(df_lastdate, "C")
D = get_tickers_by_ranking(df_lastdate, "D")

len(A), len(B), len(C), len(D),


In [ ]:

start='2020-01-01'
end='2020-06-18'

if A.any():
    close_price_A = prices(assets=A,start=start, end=end,)
    close_price_A.pct_change().mean(axis=1).cumsum().plot(label="A", legend=True, color="#0080E2")

if B.any():
    close_price_B = prices(assets=B,start=start, end=end,)
    close_price_B.pct_change().mean(axis=1).cumsum().plot(label="B", legend=True, color="#80E200")

if C.any():
    close_price_C = prices(assets=C,start=start, end=end,)
    close_price_C.pct_change().mean(axis=1).cumsum().plot(label="C", legend=True, color="#630038")
        
if D.any():        
    close_price_D = prices(assets=D,start=start, end=end,)
    close_price_D.pct_change().mean(axis=1).cumsum().plot(label="D", legend=True, color="#E28000")

    
close_price_SPY = prices(assets="SPY",start=start, end=end,)
close_price_SPY.pct_change().cumsum().plot(label="SPY", legend=True, linewidth=3, color='#e4007f')



In [ ]:
close_price_A.apply(np.log).diff().cumsum().plot(colormap="Set1") # [[symbols("TWLO"),symbols("ZNGA"),symbols("TTGT")]]
close_price_B.apply(np.log).diff().cumsum().plot(colormap="Set1")

close_price_C.apply(np.log).diff().cumsum().plot(colormap="Set1")
close_price_D.apply(np.log).diff().cumsum().plot(colormap="Set1")


In [ ]:
target = close_price_C

by_better = target.apply(np.log).diff().cumsum().iloc[-1].sort_values(ascending=False).index[:5]
target.apply(np.log).diff().cumsum()[by_better].plot(colormap="Set1")

In [ ]:
C.map(lambda x: [ x.symbol, x.asset_name])